<a href="https://www.kaggle.com/code/shyamgupta196/efficient-net-transfer-learning?scriptVersionId=184591643" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [42]:
!pip install pycocotools


^C
ERROR: Operation cancelled by user


In [43]:
!pip install efficientnet-pytorch

^C
ERROR: Operation cancelled by user


In [44]:
import os
import random
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm
from pycocotools.coco import COCO
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
%matplotlib inline
from efficientnet_pytorch import EfficientNet


In [51]:
import sys
import wandb
sys.path.append('/kaggle/usr/lib/wandblogin/')
from wandblogin import login
login()

# # Initialize wandb
wandb.init(project="fracatlas--transferlearning")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
coco = COCO('/kaggle/input/fracatlas-original-dataset/FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json')
img_dir = '/kaggle/input/fracatlas-original-dataset/FracAtlas/images/Fractured'
image_id = 12

img = coco.imgs[image_id]


In [ ]:
image = np.array(Image.open(os.path.join(img_dir, img['file_name'])))
plt.imshow(image, interpolation='nearest')
plt.show()


In [ ]:
plt.imshow(image)
cat_ids = coco.getCatIds()
anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
anns = coco.loadAnns(anns_ids)
coco.showAnns(anns)

In [ ]:
mask = coco.annToMask(anns[0])
for i in range(len(anns)):
    mask += coco.annToMask(anns[i])

plt.imshow(mask,cmap='gray')

In [ ]:

coco = COCO('/kaggle/input/fracatlas-original-dataset/FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json')

img_dir = '/kaggle/input/fracatlas-original-dataset/FracAtlas/images/Fractured'
output_dir = '/kaggle/working/fracatlas/images/fractured_masks/'

os.makedirs(output_dir, exist_ok=True)

image_ids = coco.getImgIds()
num_images =14
for i in range(num_images):    
    img = coco.loadImgs(image_ids[i])[0]
    image = np.array(Image.open(os.path.join(img_dir, img['file_name'])))

    

    cat_ids = coco.getCatIds()
    anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
    anns = coco.loadAnns(anns_ids)

    mask = coco.annToMask(anns[0])
    for j in range(1, len(anns)):
        mask += coco.annToMask(anns[j])
        
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Original Image')
    
    
    plt.subplot(1, 3, 2)
    plt.imshow(image)
    coco.showAnns(anns)
    plt.axis('off')
    plt.title('Annotations')

    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    plt.title('Combined Mask')
    
    plt.tight_layout()
    plt.show()


In [ ]:



image_folder_fractured = '/kaggle/input/fracatlas-original-dataset/FracAtlas/images/Fractured'
image_folder_non_fractured = '/kaggle/input/fracatlas-original-dataset/FracAtlas/images/Non_fractured'
train_folder = '/kaggle/working/train'
val_folder = '/kaggle/working/val'
test_folder = '/kaggle/working/test'

os.makedirs(train_folder + '/fractured', exist_ok=True)
os.makedirs(train_folder + '/non_fractured', exist_ok=True)
os.makedirs(val_folder + '/fractured', exist_ok=True)
os.makedirs(val_folder + '/non_fractured', exist_ok=True)
os.makedirs(test_folder + '/fractured', exist_ok=True)
os.makedirs(test_folder + '/non_fractured', exist_ok=True)

def split_images(image_folder, train_folder, val_folder, test_folder, split_ratio=(0.8, 0.1, 0.1)):
    image_files = os.listdir(image_folder)
    random.shuffle(image_files)
    num_images = len(image_files)
    num_train = int(split_ratio[0] * num_images)
    num_val = int(split_ratio[1] * num_images)

    train_files = image_files[:num_train]
    val_files = image_files[num_train:num_train + num_val]
    test_files = image_files[num_train + num_val:]

    for file in train_files:
        shutil.copy(os.path.join(image_folder, file), os.path.join(train_folder, file))
    for file in val_files:
        shutil.copy(os.path.join(image_folder, file), os.path.join(val_folder, file))
    for file in test_files:
        shutil.copy(os.path.join(image_folder, file), os.path.join(test_folder, file))

split_images(image_folder_fractured, train_folder + '/fractured', val_folder + '/fractured', test_folder + '/fractured')
split_images(image_folder_non_fractured, train_folder + '/non_fractured', val_folder + '/non_fractured', test_folder + '/non_fractured')

transform = transforms.Compose([
    transforms.Resize((224,224)),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_folder, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

val_dataset = torchvision.datasets.ImageFolder(val_folder, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

test_dataset = torchvision.datasets.ImageFolder(test_folder, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
import gc
gc.collect()

torch.cuda.empty_cache()

In [ ]:

model = EfficientNet.from_pretrained('efficientnet-b6')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 2)  # Assuming binary classification

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


In [47]:
roc_auc_score(all_labels, all_predictions)

0.5029769258827821

In [52]:
from PIL import ImageFile
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import wandb

ImageFile.LOAD_TRUNCATED_IMAGES = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
best_val_acc = 0.0

# Log initial configuration
wandb.config = {
    "learning_rate": 0.001,
    "epochs": 13,
    "batch_size": 150
}

num_epochs = 13
for epoch in range(num_epochs):
    model.train()
    torch.cuda.empty_cache()
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'), 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Log training metrics
    wandb.log({"epoch": epoch+1, "train_loss": running_loss/len(train_loader)})

    model.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in val_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            # Log validation images
            if i == 0:
                wandb.log({"examples": [wandb.Image(img) for img in images.cpu()]})

    val_acc = correct / total
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')
    cm = confusion_matrix(all_labels, all_predictions)
    auc_roc = roc_auc_score(all_labels, all_predictions)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Validation Accuracy: {val_acc*100:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, AUC ROC: {auc_roc:.4f}')

    # Log validation metrics
    wandb.log({
        "val_accuracy": val_acc,
        "val_precision": precision,
        "val_recall": recall,
        "val_f1_score": f1,
        "val_auc_roc": auc_roc,
        "val_confusion_matrix": wandb.plot.confusion_matrix(probs=None,
                                                             y_true=all_labels, 
                                                             preds=all_predictions,
                                                             class_names=[str(i) for i in range(outputs.size(1))])
    })

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        print("saving...")
        torch.save(model.state_dict(), '/kaggle/working/best_model.pth')

print('Finished Training')


Epoch 1/20: 100%|██████████| 244/244 [02:14<00:00,  1.81it/s]


Epoch 1/20, Loss: 0.3173, Validation Accuracy: 67.23%
Precision: 0.6383, Recall: 0.7395, F1 Score: 0.6162, AUC ROC: 0.7395
saving...


Epoch 2/20: 100%|██████████| 244/244 [02:14<00:00,  1.82it/s]


Epoch 2/20, Loss: 0.2589, Validation Accuracy: 88.47%
Precision: 0.9389, Recall: 0.6654, F1 Score: 0.7160, AUC ROC: 0.6654
saving...


Epoch 3/20: 100%|██████████| 244/244 [02:14<00:00,  1.82it/s]


...

Epoch 11/20, Loss: 0.0705, Validation Accuracy: 97.67%
Precision: 0.9863, Recall: 0.9323, F1 Score: 0.9568, AUC ROC: 0.9323


Epoch 12/20: 100%|██████████| 244/244 [02:14<00:00,  1.81it/s]


Epoch 12/20, Loss: 0.0482, Validation Accuracy: 99.09%
Precision: 0.9914, Recall: 0.9767, F1 Score: 0.9839, AUC ROC: 0.9767
saving...


Epoch 13/20:  19%|█▉        | 47/244 [00:25<01:48,  1.82it/s]


KeyboardInterrupt: 

In [ ]:


model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
model.eval()
correct = 0
total = 0
all_labels = []
all_predictions = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

test_accuracy = correct / total
precision = precision_score(all_labels, all_predictions, average='macro')
recall = recall_score(all_labels, all_predictions, average='macro')
f1 = f1_score(all_labels, all_predictions, average='macro')
cm = confusion_matrix(all_labels, all_predictions)
auc_roc = roc_auc_score(all_labels, all_predictions)

print(f'Test Accuracy: {test_accuracy*100:.2f}%')
print(f'Test Precision: {precision:.4f}, Test Recall: {recall:.4f}, Test F1 Score: {f1:.4f}, Test AUC ROC: {auc_roc:.4f}')

# Log final test metrics
wandb.log({
    "test_accuracy": test_accuracy,
    "test_precision": precision,
    "test_recall": recall,
    "test_f1_score": f1,
    "test_auc_roc": auc_roc,
    "test_confusion_matrix": wandb.plot.confusion_matrix(probs=None,
                                                          y_true=all_labels, 
                                                          preds=all_predictions,
                                                          class_names=[str(i) for i in range(outputs.size(1))])
})

wandb.finish()


Test Accuracy: 98.18%
Test Precision: 0.9770, Test Recall: 0.9606, Test F1 Score: 0.9686, Test AUC ROC: 0.9606
